In [ ]:
import dbmanager

In [ ]:
db = dbmanager.DBManager()
db.init_db()

In [ ]:
for user in db.query_users():
    print(user.name + ' ' + user.surname)

In [ ]:
for user in db.query_users_by_pref('Br', 'Al'):
    print(user.name + ' ' + user.surname)

In [ ]:
from faker import Faker
from collections import defaultdict
fake = Faker()

In [ ]:
from datetime import datetime 

start_time = datetime.now() 

fake_data = defaultdict(list)
for _ in range(1000000):
    fake_data["first_name"].append( fake.first_name() )
    fake_data["last_name"].append( fake.last_name() )
    
time_elapsed = datetime.now() - start_time

print('time elapsed {}'.format(time_elapsed))

In [ ]:
start_time = datetime.now() 
for ind in range(len(fake_data["first_name"])):
    db.add_user_short(fake_data["first_name"][ind], fake_data["last_name"][ind])
    
    if ind%1000 == 0:
        time_elapsed = datetime.now() - start_time
        print(str(ind) + ', time elapsed {}'.format(time_elapsed))

In [1]:
import mysql.connector

DB_USER = "root"
DB_NAME = "social_net"
DB_PASSWORD = "root"
DB_HOST = "db"

connection = mysql.connector.connect(
    user=DB_USER, 
    password=DB_PASSWORD,
    host=DB_HOST,
    database=DB_NAME,
    auth_plugin='mysql_native_password'
)
cursor = connection.cursor()

name_pref = 'Br'
surname_pref = 'Al'

SQL = "EXPLAIN ANALYZE SELECT id, name, surname FROM users where name like '{}%' and surname like '{}%' order by id asc".format(name_pref, surname_pref)

cursor.execute(SQL)
res = cursor.fetchall()

In [2]:
res

[("-> Filter: ((users.`name` like 'Br%') and (users.surname like 'Al%'))  (cost=92129.40 rows=11301) (actual time=0.039..356.626 rows=295 loops=1)\n    -> Index scan on users using PRIMARY  (cost=92129.40 rows=915604) (actual time=0.032..269.298 rows=1000000 loops=1)\n",)]

In [10]:
res

[("-> Sort: users.id  (cost=30830.66 rows=68512) (actual time=150.495..150.524 rows=295 loops=1)\n    -> Filter: ((users.`name` like 'Br%') and (users.surname like 'Al%'))  (actual time=0.440..150.255 rows=295 loops=1)\n        -> Index range scan on users using name_surname_pref_order_by_id  (actual time=0.425..145.067 rows=34339 loops=1)\n",)]

In [ ]:
SQL = "DROP INDEX name_surname_pref_order_by_id ON users"

cursor.execute(SQL)

connection.commit()

In [5]:
SQL = "CREATE INDEX name_surname_pref_order_by_id ON users (name(5),surname(5),id)"

cursor.execute(SQL)

connection.commit()

In [8]:
SQL = "SHOW INDEX FROM users"

cursor.execute(SQL)
res = cursor.fetchall()
res

[('users',
  0,
  'PRIMARY',
  1,
  'id',
  'A',
  915604,
  None,
  None,
  '',
  'BTREE',
  '',
  '',
  'YES',
  None),
 ('users',
  0,
  'email',
  1,
  'email',
  'A',
  1,
  None,
  None,
  'YES',
  'BTREE',
  '',
  '',
  'YES',
  None),
 ('users',
  1,
  'name_surname_pref_order_by_id',
  1,
  'name',
  'A',
  526,
  5,
  None,
  'YES',
  'BTREE',
  '',
  '',
  'YES',
  None),
 ('users',
  1,
  'name_surname_pref_order_by_id',
  2,
  'surname',
  'A',
  292050,
  5,
  None,
  'YES',
  'BTREE',
  '',
  '',
  'YES',
  None),
 ('users',
  1,
  'name_surname_pref_order_by_id',
  3,
  'id',
  'A',
  915604,
  None,
  None,
  '',
  'BTREE',
  '',
  '',
  'YES',
  None)]